In [1]:
import os
import pandas as pd
from kaggle.api.kaggle_api_extended import KaggleApi
import tensorflow as tf
import tensorflow_ranking as tfr
import numpy as np
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler

2025-02-25 19:43:41.535249: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-25 19:43:41.619327: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-25 19:43:41.619422: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-25 19:43:41.622594: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-25 19:43:41.638300: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-25 19:43:41.639786: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
data_dir = "data"
race_result_csv     = os.path.join(data_dir, "prepared.csv")
df_race_result   = pd.read_csv(race_result_csv)

print(len(df_race_result))
df_race_result.head()

1560664


,レースID,レース日付,距離(m),着順,馬名,馬齢,斤量,単勝,馬体重,競馬場名_中京,...,人気_15.0,人気_16.0,人気_17.0,人気_18.0,人気_19.0,人気_20.0,人気_21.0,人気_22.0,人気_23.0,人気_24.0
0,202102010902,2021-07-31,0.414163,7.0,バンブトンローズ,0.171856,0.347738,0.481574,0.559201,0,...,0,0,0,0,0,0,0,0,0,0
1,202102010911,2021-07-31,0.541041,4.0,ヒシヴィクトリー,0.506159,0.393765,0.367208,0.608916,0,...,0,0,0,0,0,0,0,0,0,0
2,202102010910,2021-07-31,0.142285,12.0,エムオーシャトル,0.171856,0.253119,0.364726,0.474759,0,...,0,0,0,0,0,0,0,0,0,0
3,202102010910,2021-07-31,0.142285,13.0,マリノディアナ,0.506159,0.393765,0.683876,0.461340,0,...,1,0,0,0,0,0,0,0,0,0
4,202102010910,2021-07-31,0.142285,14.0,ラキ,0.305157,0.393765,0.564725,0.507797,0,...,0,0,0,0,0,0,0,0,0,0


４次元テンソルとして学習データをまとめる。\
学習データ単位：１レース（出走馬 × 直近5レースの特徴量）

In [ ]:
race_ids = df_race_result['レースID'].unique()
tensor_list = []

horse_df = df_race_result
for i, race_id in enumerate(race_ids):
    race_df = df_race_result[df_race_result['レースID'] == race_id].copy()
    horse_names = race_df['馬名'].unique()
    race_date = race_df[race_df['馬名'] == horse_names[0]]['レース日付'].iloc[0]
    level2 = []
    for j, horse_name in enumerate(horse_names):
        horse_df = df_race_result[df_race_result['馬名'] == horse_name].copy()
        horse_df.reset_index(drop=True, inplace=True)
        unique_id = horse_df.loc[horse_df['レース日付'] == race_date].index[0]
        horse_df.drop(['レースID', 'レース日付', '馬名', '着順'], axis=1, inplace=True)
        # print(unique_id)
        level3 = []
        for k in range(5):
            if unique_id + k <= len(horse_df) - 1:
                record = horse_df.loc[k]
                record_as_list = record.apply(lambda x: int(x) if isinstance(x, bool) else x).tolist()
                # print(record_as_list)
                # print(record)
                level3.append(record_as_list)
            else:
                level3.append(0)
            # break
        level2.append(level3)
        # break
    tensor_list.append(level2)
    # break

horse_df.head()

In [ ]:
print(len(tensor_list), len(tensor_list[0]), len(tensor_list[0][0]), len(tensor_list[0][0][0]))
print(len(df_race_result))

In [ ]:
pop_list = []
for i in range(len(tensor_list)):
    for j in range(len(tensor_list[i])):
        if tensor_list[i][j][4] == 0:
            pop_list.append(i)
            break

filtered_list = [item for idx, item in enumerate(tensor_list) if idx not in pop_list]
print(len(pop_list), len(filtered_list), len(filtered_list[0]), len(filtered_list[0][0]), len(filtered_list[0][0][0]))